In [1]:
import numpy as np
import torch

print(torch.__version__)
print(torch.cuda.is_available())

1.6.0
True


# Problem
#### Given

| **Region** 	| **Temp (F)** 	| **Rainfall (mm)** 	| **Humidity (%)** 	| **Apple (ton)** 	| **Oranges (ton)** 	|
|------------	|--------------	|-------------------	|------------------	|-----------------	|-------------------	|
| Kanto      	| 73           	| 67                	| 43               	| 56              	| 70                	|
| Johto      	| 91           	| 88                	| 64               	| 81              	| 101               	|
| Hoenn      	| 87           	| 134               	| 58               	| 119             	| 133               	|
| Sinnoh     	| 102          	| 43                	| 37               	| 22              	| 37                	|
| Unova      	| 69           	| 96                	| 70               	| 103             	| 119               	|

#### Task
- Build a Regression model to predict apple and orange yield as

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

# Solution
## 1. Prepare data

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]], dtype='float32')


# Targets (apples, oranges)
targets = np.array([
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]], dtype='float32')

In [3]:
# Convert inputs and targets to tensors
X = torch.from_numpy(inputs)
y = torch.from_numpy(targets)

print(X)
print(y)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## 2. Linear Regression from scratch
#### 2.1 Model
- Define model: $y = X*W^T + b$

- X = Input data

$\begin{bmatrix}
73 & 67 & 43 \\
91 & 88 & 64 \\
... & ... & ... \\
69 & 96 & 70
\end{bmatrix}$

- $W^T$: W = (2,3) tensor

$\begin{bmatrix}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23} \\
\end{bmatrix}$

- b = (2,) tensor

$\begin{bmatrix}
b_1 & b_2 \\
b_1 & b_2 \\
... & ... \\
b_1 & b_2 \\
\end{bmatrix}$



In [4]:
def mse(t1, t2):
    '''MSE Loss'''
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

def fit(X,y):
    # Init W, b
    W = torch.randn(2, 3, requires_grad=True)
    b = torch.randn(2, requires_grad=True)

    # Train for 1000 epochs
    for i in range(1000):
        # Forward: Model
        y_ = X @ W.t() + b
        loss = mse(y_, y)
        
        # Backprop - Grad descent: Optimizer
        loss.backward()
        with torch.no_grad():
            # Update W,b
            W -= W.grad * 1e-5
            b -= b.grad * 1e-5

            # Reset Grad (=0)
            W.grad.zero_()
            b.grad.zero_()

        # Print the progress
        if (i+1)%100==0:
            print("Epoch {}: loss = {}".format(str(i+1), loss))

    return W, b

#### 2.2 Train

In [5]:
# Train and get final model
W, b = fit(X,y)

Epoch 100: loss = 540.4302978515625
Epoch 200: loss = 212.7572784423828
Epoch 300: loss = 110.46502685546875
Epoch 400: loss = 73.1790542602539
Epoch 500: loss = 55.596466064453125
Epoch 600: loss = 44.755897521972656
Epoch 700: loss = 36.813533782958984
Epoch 800: loss = 30.51938819885254
Epoch 900: loss = 25.38027572631836
Epoch 1000: loss = 21.14005470275879


#### 2.3 Test result

In [6]:
# Suppose test set = train set
X_test = X
y_test = y

# Predict and calculate loss
y_test_ = X_test @ W.t() + b
loss = mse(y_test_, y_test)

print(y_test_)
print(loss)

tensor([[ 57.4870,  70.9981],
        [ 79.6267,  96.8389],
        [124.0437, 140.5685],
        [ 22.7137,  39.2817],
        [ 96.4704, 111.4148]], grad_fn=<AddBackward0>)
tensor(21.1016, grad_fn=<DivBackward0>)


## 3. Linear Regression by PyTorch

In [7]:
from torch.utils.data import TensorDataset

# Add X, y as torch dataset
train_ds = TensorDataset(X, y)

In [8]:
from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

#### 3.1 Models

In [9]:
import torch.nn as nn

# Define model
lr_model = nn.Linear(3, 2)

# Check model params
print(lr_model.weight)
print(lr_model.bias)

Parameter containing:
tensor([[ 0.1650,  0.2156,  0.5539],
        [ 0.3330, -0.3646, -0.3487]], requires_grad=True)
Parameter containing:
tensor([-0.4029,  0.2775], requires_grad=True)


In [10]:
import torch.nn.functional as F

# Loss function: pytorch MSE
mse_fn = F.mse_loss

In [11]:
# Optimizer: SGD
sgd_opt = torch.optim.SGD(lr_model.parameters(), lr=1e-5)

In [12]:
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for Xb,yb in train_dl:
            
            # 1. Generate predictions
            yb_ = model(Xb)
            
            # 2. Calculate loss
            loss = loss_fn(yb_, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

#### 3.2 Train

In [13]:
fit(
    num_epochs=1000,
    model=lr_model,
    loss_fn=mse_fn,
    opt=sgd_opt,
    train_dl=train_dl)

Epoch [100/1000], Loss: 254.4128
Epoch [200/1000], Loss: 87.7300
Epoch [300/1000], Loss: 38.2366
Epoch [400/1000], Loss: 22.0911
Epoch [500/1000], Loss: 15.6879
Epoch [600/1000], Loss: 12.3376
Epoch [700/1000], Loss: 10.1086
Epoch [800/1000], Loss: 8.4138
Epoch [900/1000], Loss: 7.0511
Epoch [1000/1000], Loss: 5.9328


#### 3.3 Test

In [14]:
# Suppose test set = train set
X_test = X
y_test = y

# Predict and calculate loss
y_test_ = lr_model(X_test)
loss = mse_fn(y_test_, y_test)

print(y_test_)
print(loss)

tensor([[ 57.1149,  70.6505],
        [ 82.2059,  98.3332],
        [118.7748, 137.7253],
        [ 21.1884,  38.4705],
        [101.7760, 114.2781]], grad_fn=<AddmmBackward>)
tensor(5.9226, grad_fn=<MseLossBackward>)
